In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)


tensor([[-0.0708, -0.0593, -0.0938, -0.0838,  0.0861,  0.2828,  0.1193,  0.1542,
          0.2265,  0.0179],
        [-0.2309,  0.0375,  0.0825, -0.0625,  0.0230,  0.1233,  0.2057,  0.0978,
          0.0551,  0.2728]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))


In [3]:
net = MLP()
net(X)


tensor([[-0.0496, -0.0140, -0.2431,  0.1145, -0.1399, -0.0493, -0.1405, -0.1147,
          0.0794,  0.0311],
        [-0.0599, -0.1118, -0.2627,  0.0385, -0.0174, -0.1348, -0.0338, -0.0840,
          0.0552,  0.1975]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X


In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[-0.1837,  0.0175, -0.3518,  0.1078,  0.0816,  0.1040, -0.2392,  0.0979,
          0.1543,  0.1708],
        [-0.2314,  0.0672, -0.1779,  0.1102, -0.0449, -0.0293, -0.0047,  0.0076,
          0.1789, -0.0075]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [7]:
net = FixedHiddenMLP()
net(X)


tensor(-0.0293, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(0.0917, grad_fn=<SumBackward0>)

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)


tensor([[-0.2019],
        [-0.2557]], grad_fn=<AddmmBackward0>)

In [10]:
print(net[2].state_dict())


OrderedDict([('weight', tensor([[-0.3209,  0.0685, -0.1600,  0.1868, -0.0790, -0.0452,  0.2118,  0.2394]])), ('bias', tensor([-0.0808]))])


In [11]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0808], requires_grad=True)
tensor([-0.0808])


In [12]:
net[2].weight.grad == None


True

In [13]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [14]:
net.state_dict()['2.bias'].data


tensor([-0.0808])

In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[0.0779],
        [0.0779]], grad_fn=<AddmmBackward0>)

In [16]:
print(rgnet)


Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data


tensor([ 0.2875, -0.3924,  0.0991, -0.2183, -0.2708,  0.1553,  0.4951,  0.4767])

In [18]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([-0.0224, -0.0010, -0.0072, -0.0083]), tensor(0.))

In [19]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([ 0.6200,  0.4947, -0.4522, -0.5444])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000,  8.9186, -0.0000],
        [-7.4601, -0.0000,  0.0000,  6.6675]], grad_fn=<SliceBackward0>)

In [23]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]


tensor([42.0000,  1.0000,  9.9186,  1.0000])

In [24]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [25]:
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

def get_net():
    net = nn.Sequential()
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dense(10))
    return net

net = get_net()


In [26]:
print(net.collect_params)
print(net.collect_params())


<bound method Block.collect_params of Sequential(
  (0): Dense(-1 -> 256, Activation(relu))
  (1): Dense(-1 -> 10, linear)
)>
sequential0_ (
  Parameter dense0_weight (shape=(256, -1), dtype=float32)
  Parameter dense0_bias (shape=(256,), dtype=float32)
  Parameter dense1_weight (shape=(10, -1), dtype=float32)
  Parameter dense1_bias (shape=(10,), dtype=float32)
)


In [27]:
net.initialize()
net.collect_params()

sequential0_ (
  Parameter dense0_weight (shape=(256, -1), dtype=float32)
  Parameter dense0_bias (shape=(256,), dtype=float32)
  Parameter dense1_weight (shape=(10, -1), dtype=float32)
  Parameter dense1_bias (shape=(10,), dtype=float32)
)

In [28]:
X = np.random.uniform(size=(2, 20))
net(X)

net.collect_params()


sequential0_ (
  Parameter dense0_weight (shape=(256, 20), dtype=float32)
  Parameter dense0_bias (shape=(256,), dtype=float32)
  Parameter dense1_weight (shape=(10, 256), dtype=float32)
  Parameter dense1_bias (shape=(10,), dtype=float32)
)

In [29]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()


In [30]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))


tensor([-2., -1.,  0.,  1.,  2.])

In [31]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())


In [32]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())


In [33]:
Y = net(torch.rand(4, 8))
Y.mean()


tensor(-1.0477e-09, grad_fn=<MeanBackward0>)

In [34]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


In [35]:
linear = MyLinear(5, 3)
linear.weight


Parameter containing:
tensor([[-0.1166, -0.3888,  0.8579],
        [-2.3237, -1.8812, -0.9979],
        [ 0.4485, -1.7661, -1.3030],
        [ 0.5095, -0.0528, -1.0793],
        [ 0.0063,  0.6068, -0.2132]], requires_grad=True)

In [36]:
linear(torch.rand(2, 5))


tensor([[0.0000, 0.0000, 2.0690],
        [0.0000, 0.0000, 0.3751]])

In [37]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))


tensor([[ 9.5172],
        [10.2037]])

In [38]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')


In [39]:
x2 = torch.load('x-file')
x2


tensor([0, 1, 2, 3])

In [40]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [41]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [42]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)


In [43]:
torch.save(net.state_dict(), 'mlp.params')


In [44]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()


MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [45]:
Y_clone = clone(X)
Y_clone == Y


tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [46]:
!nvidia-smi


Tue May 20 14:49:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.09                 Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   67C    P0             30W /  115W |     886MiB /   6144MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [47]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')


(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [48]:
torch.cuda.device_count()


1

In [49]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()


(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [50]:
x = torch.tensor([1, 2, 3])
x.device


device(type='cpu')

In [51]:
X = torch.ones(2, 3, device=try_gpu())
X


tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [52]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y


tensor([[0.5399, 0.6908, 0.8689],
        [0.9580, 0.2778, 0.7509]])

In [54]:
Z = X.cuda(1)
print(X)
print(Z)


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [55]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())


In [56]:
net[0].weight.data.device


device(type='cuda', index=0)